In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from string import punctuation, whitespace
from ast import literal_eval
import regex as re

import os

In [5]:
cur_directory = os.path.abspath('')

relative_input_directory = 'outputs/intermediate/'
data_input_path = os.path.join(cur_directory, relative_input_directory)


In [6]:
# Manually added in the missing text chunk (xlsx decided that the chunk was a giant URL)
chunk = pd.read_excel(os.path.join(data_input_path, "df_chunk 12.xlsx"))
long = pd.read_excel(os.path.join(data_input_path, "gpt_responses_long_final_clean.xlsx"))

In [7]:
# Determine if there are any IDs in the chunk file that didn't make it into the long file
missing_lst = [x for x in chunk['text_chunk_id'] if x not in long['text_chunk_id'].unique()]
print(missing_lst)

['ACF-2023-0011-0485-0']


In [8]:
missing_df = pd.DataFrame()
for tid in missing_lst:
    #add_one = chunk[chunk['text_chunk_id']=='ACF-2023-0011-0485-0']
    add_one = chunk[chunk['text_chunk_id'] == tid]
    add_one['topic_number'] ="Missing Segment"
    add_one['topic'] ="uncategorized"
    add_one['segment_number']="None"
    add_one['segment']= ""
    add_one['intent_number'] ="uncategorized"
    add_one['intent']="uncategorized"
    missing_df = pd.concat([missing_df, add_one])
add_one.columns

Index(['Unnamed: 0', 'Document', 'chunk', 'to_translate', 'all_text',
       'validation', 'dont_run', 'Comment_counts', 'attachment_text_counts',
       'text', 'text_chunk_id', 'topic_number', 'topic', 'segment_number',
       'segment', 'intent_number', 'intent'],
      dtype='object')

In [9]:
long = long[['text_chunk_id', 'Document', 'text',
       'to_translate', 'validation', 'dont_run', 'Comment_counts',
       'attachment_text_counts', 'topic_number', 'topic', 'segment_number',
       'segment', 'intent_number', 'intent']]
missing_df = missing_df[['text_chunk_id', 'Document', 'text',
       'to_translate', 'validation', 'dont_run', 'Comment_counts',
       'attachment_text_counts', 'topic_number', 'topic', 'segment_number',
       'segment', 'intent_number', 'intent']]
long = pd.concat([long, missing_df])

In [10]:
def find_indices(long, short):
    '''
    Given a long and short input string, find the starting indices of the short
    string within the long string.
    Inputs
        long : String
        short : string
    Returns 
        list of integers
    '''
    long = long.lower()
    short = short.lower()
    return [i for i in range(len(long)) if long.startswith(short, i)]

def col_to_list(row):
    ''' 
    Convert a string list (ex: "['hello', 'hi']") into a list
    '''
    if pd.isnull(row) or row == "" or row == "[]":
        return []
    else:
        return literal_eval(row)

def find_substring(relevant_inds, clean_short, long, fuzz_function=fuzz.ratio):
    ''' 
    Given a list of indices, searches the long string to find the best match
    to the short string
    Inputs
        relevant_inds : list of starting indices for possible matches
        clean_short : string
        long : string
        fuzz_function : fuzzy matching function to use for matching
    Returns
        best_match : string
    '''
    best_match = ""
    best_similarity = 0.0
    for idx in relevant_inds:
        long_substring = long[idx:idx+len(clean_short)]
        similarity = fuzz_function(clean_short, long_substring)
        if similarity == 100:
            best_similarity = similarity
            best_match = long_substring
            break
        elif similarity > best_similarity:
            best_similarity = similarity
            best_match = long_substring
    if best_similarity < 90:
        return False
    return best_match
    
def find_best_matching_substring(long, short):
    '''
    Given a long string and a short string, uses fuzzy matching to find the 
    closest match to the short string within the long string.
    Inputs:
        long : string
        short : string
    Returns:
        best_string : string
    '''
    clean_short = short.rstrip(whitespace + punctuation)
    short_start = clean_short[:5].rstrip(whitespace + punctuation)
    relevant_inds = find_indices(long, short_start)
    if relevant_inds == []:
        short_start = clean_short[:4].rstrip(whitespace + punctuation)
        relevant_inds = find_indices(long, short_start)
        if relevant_inds == []:
            short_start = clean_short[:3].rstrip(whitespace + punctuation)
            relevant_inds = find_indices(long, short_start)
    best_string = find_substring(relevant_inds, clean_short, long)
    if not best_string:
        best_string = find_substring(relevant_inds, clean_short, long, fuzz.token_set_ratio)
    return best_string

def find_missing_segments(df1, df2):
    '''
    Given a chunked dataframe (df1) and a long dataframe(df2), determine which portions
    of the chunk text are not represented in the long dataframe
    Inputs:
        df1 : chunk dataframe
        df2 : long dataframe
    Outputs:
        df1_exp : New dataframe that includes columns indicating missing segments 
        problem_ids : List of text chunk ids where we couldn't find their matching substring
        df2 : Adjusted long dataframe that includes some columns helpful for troubleshooting
    '''
    df1['missing_segments'] = "" 
    df1['flag_missing_segments']= 0
    df1['flag_cant_find_sub'] = 0
    problem_ids = []
    for doc in df2['text_chunk_id'].unique():
        df_seg = df2[df2['text_chunk_id']==doc]
        df_chunk = df2[df2['text_chunk_id']==doc]
        concat_segs = ' '.join(set(df_seg['segment']))
        chunk = df_chunk['text'].iloc[0]
        
        fuzz_ratio = fuzz.ratio(concat_segs, chunk)
        fuzz_tokensort = fuzz.token_sort_ratio(concat_segs, chunk)
        if fuzz_ratio == 100 or fuzz_tokensort == 100:
            continue
        expanding_segs = ''
        for segment in df_seg['segment'].unique():
            if (segment.strip() in punctuation) or (chunk == 'HAVE FROM CHATGPT') or (segment in expanding_segs):
                continue
            best_substring = find_best_matching_substring(chunk, segment)
            if not best_substring:
                print(doc)
                problem_ids.append(doc)
                df2.loc[(df2['text_chunk_id']==doc) & (df2['segment']==segment), 'flag_cant_find_sub'] =1
                continue
            expanding_segs += best_substring
            expanding_segs += segment

            chunk = chunk.replace(best_substring[:-2], "HAVE FROM CHATGPT")
        
        final = chunk.split("HAVE FROM CHATGPT")
        final_clean = [x.rstrip(whitespace + punctuation) for x in final if any(let not in whitespace and let not in punctuation for let in x)]
        final_clean2 = [x for x in final_clean if (x.lower().strip(punctuation + whitespace) not in expanding_segs.lower()) and (len(x) > 10)]
        df1.loc[df1['text_chunk_id']==doc, 'missing_segments'] = str(final_clean2)
        df1.loc[df1['text_chunk_id']==doc, 'flag_missing_segments'] = 1
        df1.loc[df1['text_chunk_id']==doc, 'all_segments_apearing'] = str(concat_segs)
        fuzzy_lst = []
        for mseg in final_clean2:
            fuzzy_lst.append(fuzz.partial_ratio(mseg, concat_segs))
        df1.loc[df1['text_chunk_id']==doc, 'fuzz_ratio']= str(fuzzy_lst)

    df1['missing_segments'] = df1['missing_segments'].apply(col_to_list) #convert to list type
    df1['fuzz_ratio'] = df1['fuzz_ratio'].apply(col_to_list)
    df1_exp = df1.explode(["missing_segments","fuzz_ratio"])
    df1_exp = df1_exp[df1_exp['flag_missing_segments']==1]
    df1_exp['topic_number'] = 'Missing Segment'
    df1_exp['topic'] = 'uncategorized'
    df1_exp['segment_number'] = 'segment_1'
    df1_exp['intent_number'] = 'uncategorized'
    df1_exp['intent'] = 'uncategorized'

    return df1_exp, problem_ids, df2

In [11]:
missing_segments, segment_problem_ids, long2 = find_missing_segments(chunk, long)

ACF-2023-0011-0251-A1-0
ACF-2023-0011-0251-A1-0
ACF-2023-0011-0301-A1-9
ACF-2023-0011-0377-A3-11
ACF-2023-0011-0377-A3-11
ACF-2023-0011-0377-A3-11
ACF-2023-0011-0377-A3-12
ACF-2023-0011-0377-A3-3
ACF-2023-0011-0377-A3-4
ACF-2023-0011-0377-A3-4
ACF-2023-0011-0377-A3-6
ACF-2023-0011-0377-A3-8
ACF-2023-0011-0380-A3-15
ACF-2023-0011-0409-A1-0
ACF-2023-0011-0409-A1-1
ACF-2023-0011-0409-A1-1
ACF-2023-0011-0409-A1-2
ACF-2023-0011-0409-A1-2
ACF-2023-0011-0409-A1-2
ACF-2023-0011-0409-A1-2
ACF-2023-0011-0409-A1-2
ACF-2023-0011-0409-A1-3
ACF-2023-0011-0409-A1-4
ACF-2023-0011-0409-A1-4
ACF-2023-0011-0409-A1-5
ACF-2023-0011-0409-A1-5
ACF-2023-0011-0468-A1-0
ACF-2023-0011-0515-A1-1
ACF-2023-0011-0515-A1-6
ACF-2023-0011-0568-0
ACF-2023-0011-0586-A1-1
ACF-2023-0011-0610-A1-3
ACF-2023-0011-0630-A1-2
ACF-2023-0011-0640-A1-2
ACF-2023-0011-0640-A1-2
ACF-2023-0011-0667-A2-1
ACF-2023-0011-0726-A1-1
ACF-2023-0011-0824-A1-0
ACF-2023-0011-0913-A1-0
ACF-2023-0011-0939-A1-1
ACF-2023-0011-DRAFT-0954-A1-6
ACF-2023

In [15]:
# Save the missing segments
missing_segments.to_excel("missing_segments_only.xlsx")

In [12]:
# See how many segments we weren't able to find substring matches for
len(set(segment_problem_ids))

43

In [13]:
def find_substring(relevant_inds, clean_short, long, fuzz_function=fuzz.ratio):
    '''Given a list of relevant indices, a short string, a long string, and a matching function,
    return the best matching substring for short within long, and the starting index for that 
    substring, and the similarity score for that substring.'''
    
    best_match = ""
    best_similarity = 0.0
    best_idx = None

    for idx in relevant_inds:
        long_substring = long[idx:idx+len(clean_short)]

        similarity = fuzz_function(clean_short, long_substring)
        if similarity == 100:
            best_similarity = similarity
            best_match = long_substring
            best_idx = idx
            break
        elif similarity > best_similarity:
            best_similarity = similarity
            best_match = long_substring
            best_idx = idx

    return (best_match, best_idx, best_similarity)
    
def find_best_matching_substring(long, short):
    '''Given a long string and a short string, finds the best matching substring for the short 
    string within the long string. Finds the associated string index for that matching substring. 
    Finds the best similarity score for the associated substring'''

    clean_short = short.strip(whitespace + punctuation)
    short_start = clean_short[:5].strip(whitespace + punctuation)
    
    relevant_inds = find_indices(long, short_start)
    if relevant_inds == []:
        short_start = clean_short[:4].strip(whitespace + punctuation)
        relevant_inds = find_indices(long, short_start)
        if relevant_inds == []:
            short_start = clean_short[:3].strip(whitespace + punctuation)
            relevant_inds = find_indices(long, short_start)
    best_string, best_idx, best_sim = find_substring(relevant_inds, clean_short, long)
    if not best_string:
        best_string, best_idx, best_sim = find_substring(relevant_inds, clean_short, long, fuzz.token_set_ratio)
    return (best_string, best_idx, best_sim)

def find_segment_indices(df):
    '''Given a dataframe (df), returns the dataframe with two new columns identifing the 
    segment index number and the fuzz ratio to indicate the confidence of that index'''

    for chunk_id in df['text_chunk_id'].unique():
        chunk = df[df['text_chunk_id']==chunk_id].iloc[0]['text']
        for seg in df[df['text_chunk_id']==chunk_id]['segment']:
            if not pd.isna(seg):
                _, idx, best_sim = find_best_matching_substring(chunk, seg)
                df.loc[(df['segment']==seg) & (df['text_chunk_id']==chunk_id), 'segment_index'] = idx
                df.loc[(df['segment']==seg) & (df['text_chunk_id']==chunk_id), 'fuzz_ratio_index'] = best_sim
    return df


In [14]:
df = pd.read_excel(os.path.join(data_input_path, "gpt_responses_long_final_clean.xlsx"))
mdf = pd.read_excel("missing_segments_only.xlsx")

df = df[['Unnamed: 0', 'comment number', 'text_chunk_id', 'Document', 'text',
       'to_translate', 'validation', 'dont_run', 'Comment_counts',
       'attachment_text_counts', 'topic_number', 'topic', 'segment_number',
       'segment', 'intent_number', 'intent']]

In [15]:
# Align the long df and the missing df columns
df['fuzz_ratio'] = None
df['all_segments_apearing'] = None
df = df[['text_chunk_id', 'Document', 'text',
       'to_translate', 'validation', 'dont_run', 'Comment_counts',
       'attachment_text_counts', 'topic_number', 'topic', 'segment_number',
       'segment', 'fuzz_ratio', 'all_segments_apearing', 'intent_number', 'intent']]
mdf = mdf[['text_chunk_id', 'Document', 'text',
       'to_translate', 'validation', 'dont_run', 'Comment_counts',
       'attachment_text_counts', 'topic_number', 'topic', 'segment_number',
       'missing_segments', 'fuzz_ratio', 'all_segments_apearing', 'intent_number', 'intent']]
mdf = mdf.rename(columns={"missing_segments":"segment"})
mdf.columns

Index(['text_chunk_id', 'Document', 'text', 'to_translate', 'validation',
       'dont_run', 'Comment_counts', 'attachment_text_counts', 'topic_number',
       'topic', 'segment_number', 'segment', 'fuzz_ratio',
       'all_segments_apearing', 'intent_number', 'intent'],
      dtype='object')

In [16]:
# Assign indices for all segments
mdf = find_segment_indices(mdf)
df = find_segment_indices(df)

In [17]:
# Filter out blank segments from missing segment dataframe
mdf = mdf[~pd.isna(mdf['segment'])]

In [18]:
# Use regex to remove weird characters/short nonsense words from missing segments.
pattern = re.compile(r'..\.\n')
pattern2 = re.compile(r'^[a-zA-Z]{2}\.')
pattern3 = re.compile(r'^[a-zA-Z]{2}\?')
pattern4 = re.compile(r'^[a-zA-Z]{3}\.')
pattern4 = re.compile(r'^[a-zA-Z]{3}\?')

for i in mdf.index:
    row = mdf.loc[i]
    concat_segs = row['all_segments_apearing']
    seg = row['segment']
    sub_str_lst = pattern.findall(seg)
    sub_str_lst.extend(pattern2.findall(seg))
    sub_str_lst.extend(pattern3.findall(seg))
    sub_str_lst.extend(pattern4.findall(seg))
    for sub_str in sub_str_lst:
        seg = seg.replace(sub_str, "")
    mdf.loc[i,"segment"] = seg
    try:
        # Determine if the missing segment is likely already represented in the concatenated segments
        ratio = fuzz.partial_ratio(seg.replace("\n", " "), concat_segs)
        mdf.loc[i,"fuzz_ratio"] = ratio
    except:
        mdf.loc[i,"fuzz_ratio"] = 0
# Remove missing segments that are likely not missing at all (have a high fuzz ratio)
mdf = mdf[mdf['fuzz_ratio'] < 80] 
print(len(mdf))

1640


In [19]:
mdf.columns == df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [20]:
df = pd.concat([df, mdf])

print(len(df))

19631


In [26]:
df.to_excel("full_df_and_missing_segments_test.xlsx")
# Run manual check on blank segment indices and segment indices with very low fuzzy match ratio_index
# Below code will only work if all "segment_index" cells are filled

In [22]:
df = pd.read_excel("full_df_and_missing_segments_test.xlsx")

In [23]:
df = df.rename(columns={"topic_number":"topic_origin"})
df['rank'] = df.groupby('text_chunk_id')['segment_index'].rank(ascending=True, method='dense').astype(int)
df.drop("segment_index",axis=1)
df = df.rename(columns={"rank":"segment_index"})
df['topic_origin'] = np.where(df['topic_origin']!="Missing Segment", "ChatGPT Topic Match", df['topic_origin'])
print(df['topic_origin'].unique())
df['segment_length'] = df['segment'].apply(lambda x: len(x.split(" ")))
df['segment_lessthan_30words'] = np.where(df['segment_length']<30,1,0)
df['segment_lessthan_15words'] = np.where(df['segment_length']<15,1,0)
print(df['segment_lessthan_15words'].sum())

['ChatGPT Topic Match' 'Missing Segment']
1294


In [30]:
df.to_excel("full_df_and_missing_segments5.xlsx")